This file accesses and downloads the NOAA GSOY yearly climate dataset and unites it into a single dataframe.

In [1]:
import pandas as pd
import os
import wget

In [2]:
wget.download('https://www.ncei.noaa.gov/data/global-summary-of-the-year/archive/gsoy-latest.tar.gz')

'gsoy-latest.tar.gz'

In [3]:
import tarfile

file = tarfile.open('gsoy-latest.tar.gz')
file.extractall('./gsoy-latest')
file.close()

C:\Users\Matthew Dannenberg\AppData\Local\Temp\ipykernel_34096\1599998253.py:4: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  file.extractall('./gsoy-latest')


In [4]:
root = os.getcwd()

root += '\\gsoy-latest'
files = os.listdir(root)

all_dfs = []
for filename in files:
    if filename[0:2] == "US":
        path = root + "\\" + filename
        new_df = pd.read_csv(path)
        print(filename + ' completed')
        all_dfs.append(new_df)

df = pd.concat(all_dfs)

US009052008.csv completed
US10adam002.csv completed
US10adam004.csv completed
US10adam006.csv completed
US10adam008.csv completed
US10adam010.csv completed
US10adam017.csv completed
US10adam022.csv completed
US10adam032.csv completed
US10adam036.csv completed
US10adam051.csv completed
US10adam056.csv completed
US10ante003.csv completed
US10bann007.csv completed
US10bann009.csv completed
US10boon005.csv completed
US10boon006.csv completed
US10boon009.csv completed
US10boon010.csv completed
US10box_001.csv completed
US10box_003.csv completed
US10box_004.csv completed
US10box_006.csv completed
US10box_008.csv completed
US10box_014.csv completed
US10brow004.csv completed
US10brow008.csv completed
US10buff002.csv completed
US10buff004.csv completed
US10buff006.csv completed
US10buff007.csv completed
US10buff008.csv completed
US10buff010.csv completed
US10buff013.csv completed
US10buff015.csv completed
US10buff034.csv completed
US10buff036.csv completed
US10burt002.csv completed
US10butl003.

In [5]:
df = df.reset_index()

Now, we restrict the data to the continental US.

In [6]:
states_to_remove = ['VI', 'MP', 'AK','HI','PR','AS', 'GU']

df['STATE'] = df['NAME'].map(lambda x: x[-5:-3], na_action = 'ignore')
df = df[~df['STATE'].isin(states_to_remove)]

We now remove a large number of unnecessary (and very infrequently reported) attributes.

In [7]:
df = pd.concat([df[df.columns[:-139]],df[df.columns[-1:]]],axis=1)

Some datapoints do not have listed entries for latitude and longitude. It is simplest to remove such data immediately. Likewise, we'll restrict the year to between 1950 and 2024, inclusive.

In [8]:
df = df[~df.LATITUDE.isna()]
df = df[~df.LONGITUDE.isna()]
df = df.query('DATE >= 1950 and DATE <= 2024')

In [9]:
df = df.reset_index()

In [10]:
features_to_keep = df.columns[8:-1:2]

In [ ]:
output_df = pd.concat([df[['DATE','LATITUDE','LONGITUDE','ELEVATION','NAME']],df[features_to_keep]],axis = 1)

,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,CDSD,CLDD,DP01,DP10,DP1X,...,TAVG,TMAX,TMIN,DSND,DSNW,EMSD,EMSN,SNOW,DYFG,DYTS
0,2009,43.7333,-96.6333,482.0,"SIOUX FALLS ENVIRON. CANADA, SD US",184.3,184.3,115.0,54.0,1.0,...,6.26,12.07,0.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010,43.7333,-96.6333,482.0,"SIOUX FALLS ENVIRON. CANADA, SD US",344.1,344.1,95.0,61.0,7.0,...,7.32,12.91,1.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011,43.7333,-96.6333,482.0,"SIOUX FALLS ENVIRON. CANADA, SD US",370.3,370.3,98.0,52.0,5.0,...,7.12,13.00,1.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2012,43.7333,-96.6333,482.0,"SIOUX FALLS ENVIRON. CANADA, SD US",538.1,538.1,NaN,NaN,NaN,...,9.75,16.33,3.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013,43.7333,-96.6333,482.0,"SIOUX FALLS ENVIRON. CANADA, SD US",366.8,366.8,120.0,41.0,3.0,...,6.34,12.06,0.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
689147,2020,40.6954,-96.8541,418.2,"LINCOLN 11 SW, NE US",598.1,598.1,84.0,53.0,4.0,...,11.14,17.23,5.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
689148,2021,40.6954,-96.8541,418.2,"LINCOLN 11 SW, NE US",668.7,668.7,73.0,45.0,11.0,...,11.71,17.83,5.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN
689149,2022,40.6954,-96.8541,418.2,"LINCOLN 11 SW, NE US",724.6,724.6,73.0,40.0,6.0,...,11.04,18.11,3.98,NaN,NaN,NaN,NaN,NaN,NaN,NaN
689150,2023,40.6954,-96.8541,418.2,"LINCOLN 11 SW, NE US",662.8,662.8,79.0,49.0,6.0,...,11.93,18.53,5.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
output_df.to_csv('yearly_climate_data.csv',index=False)